# 필요한 도구 설치
# 컴퓨터가 특정 작업을 수행할 수 있도록 필요한 프로그램을 추가로 설치하는 과정입니다.

In [ ]:
#'cmake' 설치: 컴퓨터 프로그램을 빌드하거나 컴파일할 때 필요한 도구
!pip install cmake

In [ ]:
# 'kiwipiepy' 설치: 한국어 문장을 분석해 단어를 나누고 품사를 판단하는 도구
!pip install kiwipiepy

In [ ]:
#'pandas' 설치: 표(스프레드시트) 형태의 데이터를 쉽게 다루게 해주는 도구
!pip install pandas

In [ ]:
# 'PyMySQL' 설치: 데이터를 저장하는 데이터베이스(MySQL)와 연결하는 도구
!pip install PyMySQL

In [ ]:
# 'tqdm' 설치: 반복 작업의 진행 상태를 화면에 표시해주는 도구
!pip install tqdm

In [ ]:
# 'gensim' 설치: 텍스트 데이터에서 중요한 단어를 분석하고 숫자로 변환하는 도구
!pip install gensim

In [ ]:
# 필요한 도구를 사용하기 위해 프로그램에 불러옵니다.
import os  # 운영체제와 관련된 작업을 처리하기 위한 도구
import pymysql  # 데이터베이스(MySQL)와 연결하고 작업하기 위한 도구
import time  # 작업 시간 측정에 사용하는 도구
import pandas as pd  # 표 데이터를 쉽게 다룰 수 있도록 돕는 도구
import numpy as np  # 수학 계산 및 배열(숫자 데이터 처리)에 사용되는 도구
from tqdm.notebook import tqdm  # 작업 진행 상태를 화면에 표시하는 도구
from gensim.models.word2vec import Word2Vec  # 단어를 숫자로 변환해주는 도구
from kiwipiepy import Kiwi  # 한국어 문장을 분석해 단어와 품사를 나누는 도구

In [ ]:
# Kiwi라는 도구를 사용해 한국어 문장을 분석합니다.
kiwi = Kiwi()


In [ ]:
# 예제 문장을 분석해봅니다. 'tokenize'는 문장을 단어별로 나눠주는 기능입니다.
kiwi.tokenize("아버지가 방에 들어가신다 그래서 슬프다 배가 아파요")


In [ ]:
# 텍스트를 분석하고 처리하는 함수 정의
# 함수는 일종의 레시피로, 같은 작업을 반복적으로 실행할 때 사용합니다.
def preprocessingText(text):
    """
    문장을 받아서 중요한 단어들(명사, 형용사)만 추출합니다.
    """
    stems = []  # 결과를 저장할 빈 상자(리스트) 생성

    # Kiwi 도구를 사용해 문장을 단어별로 나누고 품사를 분석
    tagged_review = kiwi.tokenize(text)

    # 각 단어와 품사를 확인하면서 조건에 맞는 단어만 선택
    for token in tagged_review:
        word, pos = token.form, token.tag  # 단어와 품사(tag)를 분리
        # 단어가 2글자 이상이고, 명사(NNG, NNP)나 형용사(VA)인 경우만 선택
        if len(word) >= 2 and (pos == "NNG" or pos == "NNP" or pos == "VA"):
            stems.append(word)  # 선택된 단어를 결과 상자에 추가

    return " ".join(stems)  # 선택된 단어들을 공백으로 연결해 반환

In [ ]:
# 예제 문장을 처리하여 중요한 단어만 추출해봅니다.
preprocessingText("아버지가 방에 들어가신다 그래서 슬펐다 배가 아파요")


In [ ]:
# CSV 파일에서 영화 데이터를 읽어옵니다.
# CSV는 엑셀처럼 표 형식으로 데이터를 저장한 파일입니다.
movie_df = pd.read_csv("movie_2024.csv")

In [ ]:
# 영화 데이터를 확인해봅니다.
movie_df

In [ ]:
# 영화 줄거리 데이터의 첫 번째 내용을 확인합니다.
movie_df.loc[0, 'synopsis']

In [ ]:
# 첫 번째 줄거리 데이터를 분석해 중요한 단어만 추출해봅니다.
preprocessingText(movie_df.loc[0, 'synopsis'])

In [ ]:
# 새로운 열(Column)을 추가하고 기본값으로 'NaN'(빈 값)을 설정합니다.
movie_df["synopsis_clear"]=np.NaN

In [ ]:
movie_df

In [ ]:
#반복문의 진행 상태를 나타내는 객체
from tqdm.notebook import tqdm

sum=0
#tqdm 을 사용하였기 때문에 반복문의 진행 상태를 
#진행 바로 나타냄
for i in tqdm(range(10000000)):
  sum=sum+i

In [ ]:
#행의 개수 조회
len(movie_df)

In [ ]:
#행의 개수 조회해서 row_num에 대입
row_num = len(movie_df)

In [ ]:
row_num

In [ ]:
# 영화 데이터의 각 줄(행)을 반복하며 줄거리를 처리합니다.
for index in tqdm(range(row_num)):
    try:
        # 줄거리 데이터를 읽어옵니다.
        synopsis = movie_df.loc[index, "synopsis"]
        # 줄거리 데이터를 처리해 중요한 단어만 추출합니다.
        movie_df.loc[index, "synopsis_clear"] = preprocessingText(synopsis)
    except Exception as e:
        # 에러가 발생하면 빈 값(NaN)을 입력합니다.
        movie_df.loc[index, "synopsis_clear"] = np.NaN

In [ ]:
movie_df

In [ ]:
# 처리된 줄거리 데이터에 공백을 추가합니다.
movie_df["synopsis_clear"] =  movie_df['synopsis_clear'].astype(str) + " "

In [ ]:
movie_df.loc[0,"synopsis_clear"]

In [ ]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
movie_df["synopsis_clear"].apply(lambda data : data.split(" "))

In [ ]:
#df["synopsis_clear"].apply(lambda data : data.split(" ")) : synopsis_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
movie_df["synopsis_clear_list"] = movie_df["synopsis_clear"].apply(lambda data : data.split(" "))

In [ ]:
movie_df

In [ ]:
movie_df["synopsis_clear_list"]

In [ ]:
movie_df["synopsis_clear_list"][0]

In [ ]:
from gensim.models.word2vec import Word2Vec
# gensim의 Word2Vec을 사용해 단어를 숫자로 변환합니다.
word2vec = Word2Vec(
    movie_df["synopsis_clear_list"],  # 단어 데이터를 입력
    sg=1,  # Skip-gram 방식 사용
    vector_size=80,  # 변환된 숫자의 길이
    window=5,  # 앞뒤 5개 단어를 고려
    workers=10  # 동시에 작업할 코어 수
)

# 단어를 숫자(Vector)로 변환한 데이터 확인
# 이 데이터를 통해 컴퓨터는 단어를 숫자로 이해하고 계산할 수 있습니다.

In [ ]:
# 단어와 해당 숫자(Vector) 데이터의 순서를 조회합니다.
word2vec.wv.key_to_index

In [ ]:
# 단어만 따로 추출해 확인합니다.
word2vec.wv.key_to_index.keys()

In [ ]:
# "시작"이라는 단어의 순위를 확인합니다.
# 순위는 단어가 얼마나 자주 사용되었는지를 기준으로 결정됩니다.
word2vec.wv.key_to_index["시작"]

In [ ]:
# "자신"이라는 단어의 순위를 확인합니다.
word2vec.wv.key_to_index["자신"]

In [ ]:
# "사랑"이라는 단어와 비슷한 뜻을 가진 단어들을 찾아줍니다.
# 비슷한 단어를 추천하거나 의미를 비교하는 데 사용됩니다.
word2vec.wv.most_similar("사랑")

In [ ]:
# "가족"이라는 단어와 비슷한 뜻을 가진 단어들을 찾아줍니다.
word2vec.wv.most_similar("가족")

In [ ]:
# "사랑"이라는 단어를 숫자(Vector)로 변환한 데이터를 확인합니다.
# 이 데이터는 컴퓨터가 단어를 이해하도록 돕는 숫자 집합입니다.
word2vec.wv["사랑"]

In [ ]:
# "가족"이라는 단어를 숫자(Vector)로 변환한 데이터를 확인합니다.
word2vec.wv["가족"]

In [ ]:
#WordVector로 변환한 단어들 조회
word2vec.wv.key_to_index.keys()

In [ ]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = word2vec.wv.key_to_index.keys()

In [ ]:
# 저장된 단어 데이터를 확인합니다.
word2vec_words

# 특정 영화 데이터를 기반으로 WordVector를 계산합니다.

In [ ]:
# 첫 번째 영화 제목을 가져옵니다.
title = movie_df.loc[0, "title"]
print("title=", title)

# 줄거리 데이터를 가져옵니다.
line = movie_df.loc[0, "synopsis_clear_list"]
print("line=", line)

doc2vec = None  # 초기값 설정
count = 0  # 단어 개수를 셀 변수

# 줄거리 데이터를 한 단어씩 처리합니다.
for word in line:
    print("word=", word, end="\t")
    if word in word2vec_words:  # 단어가 WordVector에 포함된 경우
        count += 1  # 단어 개수를 하나 증가
        if doc2vec is None:  # 첫 번째 단어인 경우
            doc2vec = word2vec.wv[word]  # 단어의 Vector 값을 초기값으로 설정
        else:  # 두 번째 단어부터는 값을 더합니다.
            doc2vec = doc2vec + word2vec.wv[word]

# 모든 단어의 벡터를 더한 값을 단어 개수로 나눠 평균 벡터를 구합니다.
if doc2vec is not None:
    doc2vec = doc2vec / count

print()    
print("=" * 100)
print("title=", title, ":doc2vec=", doc2vec)
print("=" * 100)

In [ ]:

# 결과로 나온 doc2vec 데이터를 확인합니다.
doc2vec

In [ ]:
# 벡터 데이터를 문자열로 변환합니다.
string_array = doc2vec.tostring()

In [ ]:
string_array

In [ ]:
# 문자열 형태의 데이터를 다시 숫자 배열로 복원합니다.
numpy_array = np.fromstring(string_array, dtype="float32")

In [ ]:
numpy_array

In [ ]:
#행의 수 조회
len(movie_df)

In [ ]:
#행의 수를 row_num에 저장
row_num = len(movie_df)

In [ ]:
row_num 

In [ ]:
#데이터베이스 연결
# 데이터베이스 연결 설정
db = pymysql.connect(
    host='RDS 엔드포인트를 입력 합니다',  # 데이터베이스 서버 주소
    port=3306,  # 데이터베이스 연결 포트
    user='admin',  # 데이터베이스 사용자 이름
    passwd='RDS 비밀 번호를 입력 합니다',  # 데이터베이스 사용자 비밀번호
    db='movie_db',  # 데이터베이스 이름
    charset='utf8'  # 데이터 인코딩 설정
)

In [ ]:
#데이터베이스 쿼리를 실행할 객체 생성
cursor = db.cursor()

In [ ]:
# 각 영화 데이터를 데이터베이스에 저장합니다.
for index in range(row_num):
    try:
        print("=" * 100)
        print("index=", index)

        # 영화 데이터를 각각 읽어옵니다.
        title = movie_df.loc[index, "title"]
        director = movie_df.loc[index, "director"]
        actor = movie_df.loc[index, "actor"]
        synopsis = movie_df.loc[index, "synopsis"]
        poster = movie_df.loc[index, "poster"]
        open_date = movie_df.loc[index, "open_date"]
        degree = movie_df.loc[index, "degree"]
        genre = movie_df.loc[index, "genre"]
        country = movie_df.loc[index, "country"]
        movie_time = movie_df.loc[index, "movie_time"]
        box_office = movie_df.loc[index, "box_office"]

        # 줄거리 데이터를 WordVector로 변환합니다.
        line = movie_df.loc[index, "synopsis_clear_list"]
        doc2vec = None
        count = 0
        for word in line:
            if word in word2vec_words:
                count += 1
                if doc2vec is None:
                    doc2vec = word2vec.wv[word]
                else:
                    doc2vec = doc2vec + word2vec.wv[word]

        if doc2vec is not None:
            doc2vec = doc2vec / count

        # SQL 쿼리를 생성하고 데이터베이스에 저장합니다.
        sql = """insert into movie_tbl 
                 (title, director, actor, synopsis, poster, open_date, degree, genre, country, movie_time, box_office, synopsis_vector) 
                 values (%s, %s, %s, %s, %s, STR_TO_DATE(%s, '%%Y.%%m.%%d.'), %s, %s, %s, %s,%s, %s)"""
        
        cursor.execute(sql, (title, director, actor, synopsis, poster, open_date, degree, genre, country, movie_time, box_office, doc2vec.tostring()))
        db.commit()
        
    except Exception as e:
        print("exception=", e)


In [ ]:
# 데이터베이스에서 특정 영화 데이터를 조회합니다.
sql = "select title,synopsis_vector from movie_tbl where num=1"
cursor.execute(sql)
rows = cursor.fetchall()

for row in rows:
    print("title:row[0]=", row[0])
    print("=" * 100)
    print("synopsis_vector:row[1]=", row[1])
    print("=" * 100)
    print("synopsis_vector:row[1]=", np.fromstring(row[1], dtype="float32"))



In [ ]:
# 데이터베이스 연결을 종료합니다.
db.close()